In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

students = pd.read_csv("students_with_clusters.csv")
mentors = pd.read_csv("mentors.csv")

## Prepare Student Feature Vector

In [2]:
student_features = students[["APS", "WWS", "PTMS", "CRS"]]

In [3]:
scaler = MinMaxScaler()
student_scaled = scaler.fit_transform(student_features)

## Encode Mentor Domain as Feature Vector

In [4]:
def mentor_domain_vector(domain):
    if domain == "Academic":
        return [1,0.3,0.5,0]
    elif domain == "Wellness":
        return [0,1,0.4,0]
    elif domain == "Career":
        return [0.2,0.2,0.4,1]
    else:
        return [0.25,0.25,0.25,0.25]

mentor_vectors = np.array(
    mentors["expertise_domain"].apply(mentor_domain_vector).tolist()
)

## Encode Mentoring Style Compatibility

In [5]:
style_mapping = {
    "At-Risk Students": "Supportive",
    "Career-Confused Students": "Strategic",
    "High Performers": "Structured"
}

students["Preferred_Style"] = students["Cluster_Label"].map(style_mapping)

## Matching Logic with Cosine Similarity + Style + Availability

In [6]:
assigned_mentors = []
matching_scores = []

for i, student_vec in enumerate(student_scaled):

    # Compute cosine similarity
    similarities = cosine_similarity(
        [student_vec], mentor_vectors
    )[0]

    # Work on a copy (important for safe iteration)
    mentor_scores = mentors.copy()

    mentor_scores["similarity"] = similarities

    # Style compatibility bonus
    preferred_style = students.iloc[i]["Preferred_Style"]

    mentor_scores["style_bonus"] = np.where(
        mentor_scores["mentoring_style"] == preferred_style,
        0.10,
        0
    )

    # Availability weighting
    mentor_scores["availability_weight"] = (
        mentor_scores["availability_hours_per_week"] /
        mentor_scores["availability_hours_per_week"].max()
    ) * 0.15

    # Final matching score
    mentor_scores["final_score"] = (
        mentor_scores["similarity"] +
        mentor_scores["style_bonus"] +
        mentor_scores["availability_weight"]
    )

    # Capacity constraint
    eligible = mentor_scores[
        mentor_scores["current_load"] < mentor_scores["max_students"]
    ]

    if len(eligible) == 0:
        assigned_mentors.append("No Available Mentor")
        matching_scores.append(0)
        continue

    # Select best mentor
    selected = eligible.sort_values(
        by="final_score",
        ascending=False
    ).iloc[0]

    # Update load in original mentors table
    mentor_index = mentors[
        mentors["mentor_id"] == selected["mentor_id"]
    ].index

    mentors.loc[mentor_index, "current_load"] += 1

    assigned_mentors.append(selected["mentor_name"])
    matching_scores.append(round(selected["final_score"], 3))

# Store results
students["Assigned_Mentor"] = assigned_mentors
students["Matching_Score"] = matching_scores


## Recommended Intervention Mapping

In [7]:
def intervention(row):

    if row["Cluster_Label"] == "At-Risk Students":
        return "Immediate Academic & Wellness Intervention"

    elif row["Cluster_Label"] == "Career-Confused Students":
        return "Career Planning & Skill Alignment"

    elif row["Cluster_Label"] == "High Performers":
        return "Advanced Growth & Leadership Coaching"

    else:
        return "Regular Mentoring"

students["Recommended_Intervention"] = students.apply(
    intervention, axis=1
)

## Alert Simulation

In [8]:
def alert(row):
    if row["SRI"] < 50:
        return "⚠ HIGH RISK ALERT"
    elif row["Cluster_Label"] == "At-Risk Students":
        return "Monitor Closely"
    else:
        return "Stable"

students["Alert_Status"] = students.apply(alert, axis=1)

## Final Recommendation Table

In [9]:
final_table = students[[
    "student_id",
    "Cluster_Label",
    "SRI",
    "Assigned_Mentor",
    "Matching_Score",
    "Recommended_Intervention",
    "Alert_Status"
]]

# Save to outputs folder (create folder if not exists)
import os
os.makedirs("outputs", exist_ok=True)

final_table.to_csv("outputs/student_mentor_recommendations.csv", index=False)

print("Mentor matching completed successfully!")

final_table.head()

Mentor matching completed successfully!


,student_id,Cluster_Label,SRI,Assigned_Mentor,Matching_Score,Recommended_Intervention,Alert_Status
0,S001,At-Risk Students,60.77,Mentor_18,1.133,Immediate Academic & Wellness Intervention,Monitor Closely
1,S002,High Performers,75.17,Mentor_1,1.015,Advanced Growth & Leadership Coaching,Stable
2,S003,Career-Confused Students,55.87,Mentor_7,0.996,Career Planning & Skill Alignment,Stable
3,S004,High Performers,64.04,Mentor_7,1.005,Advanced Growth & Leadership Coaching,Stable
4,S005,Career-Confused Students,55.18,Mentor_5,1.025,Career Planning & Skill Alignment,Stable
